# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [3]:
encoding_dim = 25
input_reg = keras.Input(shape=(vec_length,))
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(75, activation='relu')(encoded)
encoded = layers.Dense(40, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = encoded_input
decoded = layers.Dense(40, activation='relu')(decoded)
decoded = layers.Dense(75, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [4]:
autoencoder.compile(optimizer='adamax', loss='binary_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=100,
                batch_size=8,
                shuffle=True,
                validation_data=(val_vecs, val_vecs))

Epoch 1/100
1250/1250 [==============================] - 1s 893us/step - loss: 0.1986 - val_loss: 0.1788
Epoch 2/100
1250/1250 [==============================] - 1s 812us/step - loss: 0.1666 - val_loss: 0.1546
Epoch 3/100
1250/1250 [==============================] - 1s 803us/step - loss: 0.1475 - val_loss: 0.1416
Epoch 4/100
1250/1250 [==============================] - 1s 817us/step - loss: 0.1377 - val_loss: 0.1342
Epoch 5/100
1250/1250 [==============================] - 1s 809us/step - loss: 0.1307 - val_loss: 0.1294
Epoch 6/100
1250/1250 [==============================] - 1s 813us/step - loss: 0.1257 - val_loss: 0.1258
Epoch 7/100
1250/1250 [==============================] - 1s 838us/step - loss: 0.1215 - val_loss: 0.1227
Epoch 8/100
1250/1250 [==============================] - 1s 842us/step - loss: 0.1177 - val_loss: 0.1195
Epoch 9/100
1250/1250 [==============================] - 1s 846us/step - loss: 0.1148 - val_loss: 0.1177
Epoch 10/100
1250/1250 [==============================]

1250/1250 [==============================] - 1s 768us/step - loss: 0.0222 - val_loss: 0.0262
Epoch 79/100
1250/1250 [==============================] - 1s 772us/step - loss: 0.0206 - val_loss: 0.0240
Epoch 80/100
1250/1250 [==============================] - 1s 771us/step - loss: 0.0192 - val_loss: 0.0224
Epoch 81/100
1250/1250 [==============================] - 1s 769us/step - loss: 0.0178 - val_loss: 0.0213
Epoch 82/100
1250/1250 [==============================] - 1s 772us/step - loss: 0.0164 - val_loss: 0.0186
Epoch 83/100
1250/1250 [==============================] - 1s 801us/step - loss: 0.0152 - val_loss: 0.0168
Epoch 84/100
1250/1250 [==============================] - 1s 881us/step - loss: 0.0139 - val_loss: 0.0194
Epoch 85/100
1250/1250 [==============================] - 1s 855us/step - loss: 0.0130 - val_loss: 0.0145
Epoch 86/100
1250/1250 [==============================] - 1s 828us/step - loss: 0.0119 - val_loss: 0.0135
Epoch 87/100
1250/1250 [==============================] - 1

Evaluation

In [5]:
decoded_regs = autoencoder.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WVY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
accuracy = 0.961
